In [25]:
# !pip install transformers[torch]
# !pip install transformers[sentencepiece]
#! pip install accelerate -U
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4923244c476446daa2aa9bf6593d4b5dbf6e201911787545fe87acc41c84c1b1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
# import transformers
from transformers import AutoConfig
from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
#model = "gpt"

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'm

In [6]:
dataset_samsum = load_dataset("samsum")

In [7]:
# Split Lengths
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split Lengths: {split_lengths}")

Split Lengths: [14732, 819, 818]


In [8]:
# Features in dataset
print(f"Features: {dataset_samsum['test'].column_names}")

Features: ['id', 'dialogue', 'summary']


In [9]:
# what are the ids:
print("\nid:")
print(dataset_samsum["test"][0]["id"])


id:
13862856


In [10]:
# Dialogues
print("dialogue: ")
print(dataset_samsum["test"][0]["dialogue"])

dialogue: 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye


In [11]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length = 1024, truncation = True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length = 128, truncation = True)

    return {
        "input_ids" : input_encodings["input_ids"],
        "attention_mask" : input_encodings["attention_mask"],
        "labels" : target_encodings["input_ids"]
    }

In [12]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [14]:
columns = ["input_ids", "attention_mask", "labels"]
dataset_samsum_pt.set_format(type = "torch", columns = columns)

In [15]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [16]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [19]:
training_args = TrainingArguments(
    #output_dir = "D:\\PracticeProjects\\NLP_Customer_care_Summarization\\model",
    output_dir = "/content/drive/MyDrive/models/NLP_Summary_E2E/",
    num_train_epochs = 1,
    warmup_steps = 500, # warms up from 0 to the learning_rate
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    logging_steps = 10,
    #push_to_hub = True,
    evaluation_strategy = "steps",
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)


In [20]:
trainer = Trainer(model = model_pegasus,
                  args = training_args,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["test"],
                  eval_dataset = dataset_samsum_pt["validation"]
                 )

In [21]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 147.7452, 'train_samples_per_second': 5.543, 'train_steps_per_second': 0.345, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 0.9963369963369964})

In [26]:
rouge_metric = load_metric("rouge", trust_remote_code=True)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [27]:
#Evaluation
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evluate_summaries_pegasus(dataset,
                              metric,
                              model,
                              tokenizer,
                              batch_size = 16,
                              device = device,
                              column_text = "article",
                              column_summaries = "highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summaries], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):
        inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")
        summaries = model.generate(
            input_ids = inputs["input_ids"].to(device),
            attention_mask = inputs["attention_mask"].to(device),
            length_penalty = 0.8,
            max_length = 128
        )
        decoded_summaries = [tokenizer.decode(s, skip_special_tokes = True, clean_up_tokenization_spaces = True) for s in summaries]
        decoded_summaries = [d.replace("<n>"," ") for d in decoded_summaries]
        metric.add(predictions = decoded_summaries, references = target_batch)
        score = metric.compute()
        return score

In [28]:
score = evluate_summaries_pegasus(
dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
batch_size=2, column_text="dialogue", column_summaries="summary")
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

  0%|          | 0/410 [00:03<?, ?it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.293333,0.054795,0.24,0.24


In [29]:
# model_pegasus.save_pretrained("pegasus-samsum-model")
# tokenizer.save_pretrained("tokenizer")

In [34]:
# Predictions
trained_model_ckpt = "akashjoy/NLP_Customer_Care_Summary"
tokenizer = AutoTokenizer.from_pretrained("transformersbook/pegasus-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("transformersbook/pegasus-samsum")
gen_kwargs = {
    "length_penalty" : 0.8,
    "num_beams" : 8,
    "max_length" : 128
}

sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model = model, tokenizer = tokenizer)

##
print("Dialogue:")
print(sample_text)
print("Reference Summary:")
print(reference)
print("Model Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Model Summary:
Amanda can't find Betty's number. Larry called Betty last time they were at the park together. Hannah wants Amanda to text Larry instead of calling Betty.
